## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,50.94,89,23,4.54,few clouds
1,1,Victoria,HK,22.2855,114.1577,88.18,81,50,5.99,scattered clouds
2,2,Koboko,UG,3.4136,30.9599,64.80,95,42,3.65,light rain
3,3,Deputatskiy,RU,69.3000,139.9000,26.40,89,76,4.12,broken clouds
4,4,Udachnyy,RU,66.4167,112.4000,30.45,86,87,24.23,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Victoria,HK,22.2855,114.1577,88.18,81,50,5.99,scattered clouds
20,20,Dar Es Salaam,TZ,-6.8235,39.2695,77.38,69,20,10.36,few clouds
23,23,Alugan,PH,12.2188,125.4808,79.07,85,99,6.91,overcast clouds
27,27,Butaritari,KI,3.0707,172.7902,81.88,66,47,12.75,scattered clouds
29,29,Hilo,US,19.7297,-155.0900,85.28,78,0,8.05,clear sky
30,30,Banda Aceh,ID,5.5577,95.3222,79.03,79,21,13.20,few clouds
33,33,Bongouanou,CI,6.6486,-4.1992,76.87,92,82,2.93,light rain
41,41,Bengkulu,ID,-3.8004,102.2655,75.24,91,98,4.03,overcast clouds
42,42,Georgetown,MY,5.4112,100.3354,82.33,88,20,3.44,few clouds
45,45,Kyaikto,MM,17.3000,97.0167,76.69,94,100,5.44,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                252
City                   252
Country                252
Lat                    252
Lng                    252
Max Temp               252
Humidity               252
Cloudiness             252
Wind Speed             252
Current Description    252
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Victoria,HK,88.18,scattered clouds,22.2855,114.1577,
20,Dar Es Salaam,TZ,77.38,few clouds,-6.8235,39.2695,
23,Alugan,PH,79.07,overcast clouds,12.2188,125.4808,
27,Butaritari,KI,81.88,scattered clouds,3.0707,172.7902,
29,Hilo,US,85.28,clear sky,19.7297,-155.0900,
30,Banda Aceh,ID,79.03,few clouds,5.5577,95.3222,
33,Bongouanou,CI,76.87,light rain,6.6486,-4.1992,
41,Bengkulu,ID,75.24,overcast clouds,-3.8004,102.2655,
42,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
45,Kyaikto,MM,76.69,overcast clouds,17.3000,97.0167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping....")
        

Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....
Hotel not found. Skipping....


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Victoria,HK,88.18,scattered clouds,22.2855,114.1577,Mini Central
20,Dar Es Salaam,TZ,77.38,few clouds,-6.8235,39.2695,"Holiday Inn Dar Es Salaam City Centre, an IHG ..."
23,Alugan,PH,79.07,overcast clouds,12.2188,125.4808,JM Lodge
27,Butaritari,KI,81.88,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
29,Hilo,US,85.28,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
669,Florida Ridge,US,86.95,overcast clouds,27.5803,-80.3867,Econo Lodge Vero Beach - Downtown
670,Pingshan,CN,79.32,broken clouds,22.9938,114.7131,Island Skylight Resort
671,Altagracia De Orituco,VE,90.28,scattered clouds,9.8601,-66.3814,Posada Quinta Mi Refugio
676,Jaisalmer,IN,84.22,few clouds,26.9147,70.9181,Hotel Fifu


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30., 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))